# Empathy-Driven Arabic Chatbot



### Team

#### Member 1: Noor Elkindy (31-9103)

#### Member 2: Ghada Mansour (40-9240)



In [1]:
import pandas as pd
import nltk
import numpy as np
import re
import torch
import torch.nn as nn


from nltk.stem.isri import ISRIStemmer #ISRIStemmer
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#nltk.download('all')
#pip install torch torchvision==0.10.0 torchaudio===0.9.0

## Dataset Cleaning and Preparing

In [3]:
# Reading the Dataset
df = pd.read_csv('arabic dataset train_2.csv'  , encoding = 'utf-16',low_memory=False)

# Dropping unwanted Columns
df = df.drop(df.loc[:,'selfeval':], axis=1) 
df = df.drop(['conv_id','speaker_idx','prompt','utterance'], axis=1)

# Renaming Columns
df = df.rename(columns={'translated_prompt':'prompt','translated_utterance':'utterance' })
df.head(10)


,utterance_idx,context,prompt,utterance
0,1,afraid,كنت أخاف من الظلام,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام
1,2,afraid,كنت أخاف من الظلام,أجل؟ أنا حقا لا أرى كيف
2,3,afraid,كنت أخاف من الظلام,لا تشعر بذلك .. إنه لأمر عجيب
3,4,afraid,كنت أخاف من الظلام,أصطدم بالفعل بجدران فارغة في كثير من الأحيان ل...
4,5,afraid,كنت أخاف من الظلام,ظننت ذلك عمليا .. وكنت أتعرق
5,6,afraid,كنت أخاف من الظلام,انتظر ما هو العرق
6,1,afraid,في العام الماضي ، سقطت شجرة على منزلي بينما كا...,ما الفرق الذي يحدثه العام. في إحدى الأمسيات من...
7,2,afraid,في العام الماضي ، سقطت شجرة على منزلي بينما كا...,هذا مخيف جدا. آمل ألا يصاب أحد بأذى.
8,3,afraid,في العام الماضي ، سقطت شجرة على منزلي بينما كا...,كنا على ما يرام ، على الرغم من أن الشجرة اخترق...
9,4,afraid,في العام الماضي ، سقطت شجرة على منزلي بينما كا...,سعداء جدا الجميع بخير !! يمكن إصلاح كل شيء آخر.


In [4]:
# Making sure it doesn't have any Null Values
df.isnull().sum()

utterance_idx    0
context          0
prompt           0
utterance        0
dtype: int64

In [5]:
df.describe()
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   utterance_idx  5323 non-null   int64 
 1   context        5323 non-null   object
 2   prompt         5323 non-null   object
 3   utterance      5323 non-null   object
dtypes: int64(1), object(3)
memory usage: 166.5+ KB


(5323, 4)

## Preprocessing Functions

Here removing Punctuations is done before Tokenization since Punctuations in Arabic are not part of the words, unlike English

In [6]:
# Removing Punctuations, Numbers and English Words

def ignore_words(sent):
    
    arabic_punctuations = ["؛",".","،",",","!","؟","×","÷","*","%","&","#","?","$","٪","`","'",'"',"ً"] #Tanween (Nunation) was handled between the last Quotations, however it is not obvious  
    new_sent = ''
    
    sent = re.sub(r'[0-9a-zA-Z]', '', sent)
    
    for w in sent:
        if w not in arabic_punctuations:
            new_sent += w
    
    return new_sent


In [7]:
# Tokenization

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [8]:
# Stemming

isri = ISRIStemmer()

def stem(word):
    return isri.stem(word)

In [9]:
# All the needed Preprocessing

def preprocessing(sent):
    sent = ignore_words(sent)
    sent = tokenize(sent)
    sent = [stem(word) for word in sent]
    return sent

In [10]:
# Bag-of-Words

def bag_of_words(preprocessed_sentence, all_words):
    bag = np.zeros(len(all_words), dtype = np.float32)
    
    for idx, word in enumerate(all_words):
        if word in preprocessed_sentence: 
            bag[idx] = 1.0
            
    return bag

## Deep Learning Model

In [11]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU() 
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        
        return out

## Train

In [12]:
# Extracting the Input Texts only for each Conversation in the Utterance based on the Utterance ID

df_cntxt_input_utt = df.copy()
df_cntxt_input_utt.shape

for index, row in df_cntxt_input_utt.iterrows():
    if (row['utterance_idx'] % 2 == 0):
        df_cntxt_input_utt = df_cntxt_input_utt.drop(index) 
        
df_cntxt_input_utt.shape

(2772, 4)

In [13]:
df_cntxt_input_utt= df_cntxt_input_utt.drop(['utterance_idx','prompt'], axis=1)

In [14]:
# Preparing the needed Variables for the Model

all_words = []
contexts = []
xy = []


for index, item in df_cntxt_input_utt.iterrows():
    context = item['context']
    contexts.append(context)
    utterance = preprocessing(item['utterance'])
    all_words.extend(utterance)
    xy.append((utterance, context))

all_words = sorted(set(all_words)) #sorting and removing duplicates 
contexts = sorted(set(contexts)) #sorting and removing duplicates


In [15]:
# Preparing the Training Dataset

x_train = []
y_train = []

for (utterance, context) in xy:
    bag = bag_of_words(utterance, all_words)
    x_train.append(bag)
    
    label = contexts.index(context)
    y_train.append(label)
    
x_train = np.array(x_train)
y_train = np.array(y_train, dtype = np.longlong)

In [16]:
# Creating a custom Dataset

class ChatbotDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
        
    # allowing indexing the Dataset
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    # len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    

In [19]:
# Applying the Training Model on the Dataset


# Hyperparameters

batch_size = 32  #This Batch Size is chosen since it is the standard one (best one for small Datasets)
input_size = len(x_train[0]) #length of each bag_of_words, which is equals to length of all_words array
hidden_size = 32  #it is suggested to be a Value between the input_size and output_size. After Testing, 32 was the best Value
output_size = len(contexts)  #Number of different classes we have in the Contexts
num_epochs = 1000
learning_rate = 0.007 #After testing different Learning Rates, 0.007 was the best one in Training the Model

dataset = ChatbotDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for words, labels in train_loader: #Batch Data
        words = words.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # backward pass and optimzation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0338
Epoch [200/1000], Loss: 0.0398
Epoch [300/1000], Loss: 0.0001
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000


In [29]:
#saving Data

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "contexts": contexts
}

FILE = 'data.pth'
torch.save(data, FILE)

##  The Chatbot

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

FILE = 'data.pth'
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
context = data['contexts']
model_state = data['model_state']


model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval() #Eval Mode

chatbot_name = 'عاطف'
print('(اكتب "إغلاق" لإنهاء المحادثة)\n')
print('مرحباً، معك عاطف. هيا لنتحدث :)\n')

while True:
    
    sentence = input(' أنت:')

    if (ignore_words(sentence) == 'إغلاق'):
        break
    
    elif ((ignore_words(sentence) == 'أهلا') or (ignore_words(sentence) == 'مرحبا')):
        print('أهلاً بك')
            
    
    elif ((ignore_words(sentence) == 'إلى اللقاء') or (ignore_words(sentence) == 'مع السلامة')):
        print('مع السلامة :)')
        break
        
        
    else:

        original_sentence = sentence
        sentence = preprocessing(sentence)
        X = bag_of_words(sentence, all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X)
        
        
        # Predicting the Context (Sentiment)
        
        output = model(X)
        _ , predicted = torch.max(output, dim=1)
        context = contexts [predicted.item()]

        probs = torch.softmax(output, dim=1)     #applying the Softmax Function
        prob = probs[0][predicted.item()]

        if prob.item() > 0.75:

            print(context)

            available_utt = []

            for index, row in df.iterrows(): #get the input texts from the Utterance related to the predicted Context
                if (row['context'] == context):
                      if (row['utterance_idx'] % 2 == 1):
                            available_utt.append(row['utterance'])
                            
            
            # Mesausring Similarity based on tf-idf and Cosine Similarity 

            available_utt.append(original_sentence) #appending user original_sentence before any preprocessing

            tfidf_vectorizer = TfidfVectorizer(tokenizer=preprocessing)
            tfidf = tfidf_vectorizer.fit_transform(available_utt)
            similar_vector_values = cosine_similarity(tfidf[-1], tfidf)
            similar_sentence_number = similar_vector_values.argsort()[0][-2]

            matched_vector = similar_vector_values.flatten()
            matched_vector.sort()
            vector_matched = matched_vector[-2]

            if vector_matched == 0:
                print('عذراً، لم أفهم')

            else:
                for index, row in df.iterrows(): #returning the Utterance Response of the most similar Utterance Input 
                    if (row['utterance'] == available_utt[similar_sentence_number]):
                        print (chatbot_name, ': ', df['utterance'][index+1])
                      

        else:
            print('عذراً، لم أفهم')


(اكتب "إغلاق" لإنهاء المحادثة)

مرحباً، معك عاطف. هيا لنتحدث :)

 أنت:مرحبا
أهلاً بك
 أنت:لازلت أشعر بالخوف
afraid
عاطف :  ما الذي كنت خائفا منه؟
 أنت:عندما ذهبت إلى الغابة رأيت ثعبان كبير
afraid
عاطف :  هل عضك؟ هل أنت بخير؟
 أنت:أعتقد أنني محظوظة لم يحدث ذلك . كنت أشعر أنني خائفة على الرغم بأني كنت بخير
afraid
عاطف :  أنا سعيد لأنك بخير. بعضها سام ويمكن أن يقتل.
 أنت:وأيضا أتذكر عندما رأيت فأر كبير داخل المطبخ
afraid
عاطف :  ماذا فعلتم به؟ هل حاولت قتله؟
 أنت:لا أتذكر ولكنني حاولت إغلاق الباب عليه . ولكن لم أتمكن من أتصل بالصيانة بسبب تأخر الوقت
afraid
عاطف :  هذا ما كنت سأفعله. هل تم الاعتناء به؟
 أنت:نعم، وبرغم من ذلك أنا فخور فقد أنجزت الأعمال في الفناء , ولكن شعرت بالقذارة
proud
عاطف :  أنا فقط أحب ساحة العمل مع الأطفال. يقومون بمعظم العمل ونستمتع في التراب.
 أنت:يشعرني العشب بالحكة ولكن بعد الاستحمام اشعر بالراحة 
proud
عاطف :  أوه لا. الحمام هو الجانب الإيجابي لكل الأوساخ التي أعتقد. وجدت ذات مرة ثعبانًا يحفر حفرة لزرع زهرة. تركت ذلك اليوم.
 أنت:أنني أشعر بشيء من الرعب خصوصا في 